## 1. Import Library

### 1.1. Import Standard Library

In [ ]:
import csv
import glob
import codecs
import pandas as pd
import numpy as np

### 1.2. Import Library for Regular Expression

In [ ]:
import re

### 1.3. Import for Multiplication

In [ ]:
from functools import reduce

## 2. Import Data

In [ ]:
files = glob.glob('G:\\Data\\*.xls')

df = pd.DataFrame() 
for file in files:
    temp = pd.read_excel(file, encoding = 'utf-8')
    df = df.append(temp)

## 3. Clean Dataset

### 3.1. Select Columns

In [ ]:
# Confirm column labels
df.columns

In [ ]:
# Select columns that are necessary for further analysis
col_df = ['Fruit_Name_ID','Fruit_Name','Qty_Box']
df = df[col_df]

### 3.2.Remove Spaces

In [ ]:
# Remove all white spaces in the strings
df['Fruit_Name'] = df['Fruit_Name'].str.replace(' ', '')

### 3.3. Dataset Overview

In [ ]:
# Summarize descriptive statistics of the dataset
df.describe(include = ['O'])

In [ ]:
# Confirm data type of each column
df.dtypes

## 4. Define Function

### 4.1. Extract Substrings by Regular Expression
Extract numbers ***before*** units of measure or ***after*** multiplication signs.

In [ ]:
def split_it(s):
    l = []
    re_list = re.findall(r'([0-9]+|[一|二|兩|三|四|五|六|七|八|九|十]+)[盒|組|顆|粒|個|串]|[X|x|*]([0-9]+|[一|二|兩|三|四|五|六|七|八|九|十]+)',s)
    for tup in re_list:
        for i in tup:
            if i != '':
                l.append(i)
    return l

### 4.2. Convert Strings to Numeric Values

### 4.2.1. Define Function for Conversion
Reference https://blog.csdn.net/zhengdan66/article/details/78148695

In [ ]:
# Create a dictionary help convert Chinese numerals to Arabic.
common_used_numerals = {'零': 0, '一': 1, '二': 2, '兩': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, 
                        '十': 10, '百': 100}

def string_to_digits(s):
    # If "s" is a string consisted of Arabic numerals only, return an integer object from the given string.
    if s.isdigit():
        return int(s)
    # If "s" is a string consisted of Chinese numerals, proceed the following steps:
    # Suppose "s" = "三百五十二" which is equivalent to "352" in Arabic numerals to facilitate the explanation.
    else:
        # Set the initial value of "total" as 0 which will be returned with the conversion result.
        total = 0
        # Set the initial value of "r" as 1 which indicates that the conversion will be starting with "ones place."         
        r = 1

        # i = 4 3 2 1 0
        # 1st Loop：i=4, r=1, val=2, "ones place"
        # 2nd Loop：i=3, r=1, val=10, "ones place to tens place"
        # 3rd Loop：i=2, r=10, val=5, "tens place"
        # 4th Loop：i=1, r=10, val=100, "tens place to hundreds place"
        # 5th Loop：i=0, r=100, val=3, "hundreds place"
        for i in range(len(s) - 1, -1, -1):
            val = common_used_numerals.get(s[i])
            # If "s" is "ten something" then proceed the following steps which is not the case for this scenario.
            if val >= 10 and i == 0: 
                if val > r:
                    r = val
                    total = total + val
                else:
                    r = r * val
            # 2nd Loop：10>=10
            # 4th Loop：100>=10
            elif val >= 10:
                # 2nd Loop：10>1 hence r=10 which suggests that the conversion is proceeding to "tens place"
                # 4th Loop：100>10 hence r=100 which suggests that the conversion is proceeding to "hundreds place"
                if val > r:
                    r = val
                else:
                    r = r * val
            # 1st Loop：2 = 0+1*2 
            # 3rd Loop：52 = 2+10*5
            # 5th Loop：352 = 52+100*3
            else:
                total = total + r * val
        return total

### 4.2.2. Apply Defined Function to Every Element in the List

In [ ]:
def to_num(l):    
    if l != []:
        return list(map(string_to_digits, l))
    else:
        return [1]

### 4.3. Multiply All Elements in the List

In [ ]:
def multiply_list(l):
    # If there is no unit of measure to be found, then return 1, 
    # which suggests that Qty_Piece remain the same as Qty_Box.
    if l == []:
        return 1
    return reduce(lambda x, y: x*y, l)

## 5. Data Analysis

In [ ]:
# Suppose the goal is to dspecify how many "apples" were purchased during a given period.
df_select = df[df['Fruit_Name_ID'].str.contains('APPL')]

In [ ]:
# Suppose that there is a misclassification of "products of Apple Inc." into "apple as fruit" 
# and therefore should be removed from the selected dataframe. 
df_select = df_select[df_select['Fruit_Name'].str.contains(r'^((?!iP).)*$')]

In [ ]:
# 1. Get the row count of the dataframe.
print(df_select.shape)
# 2. Make a copy of the dataframe.
res = df_select.copy()
# 3.1. Convert Qty_Box to smaller units by extracting substrings via regular expression.
res['Unit_Char'] = res['Fruit_Name'].apply(split_it)
# 3.2. Apply function that is to "convert strings to numeric values" to every element in the list of extracted substrings.
res['Unit_Num'] = res['Unit_Char'].apply(to_num)
# 4. Multiply all elements in the list.
res['Unit_Total'] = res['Unit_Num'].apply(multiply_list)
# 5. Calculate purchasing volume by smallest unit of measurement.
res['Qty_Piece'] = res['Qty_Box'] * res['Unit_Total']
# 6. Sum up purchasing volume.
sum(list(res['Qty_Piece']))